In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [3]:

import tensorflow_datasets as tfds

# Load the Food101 dataset with 'train' split
(ds_train, ds_test) = tfds.load('food101', split='train', as_supervised=True, with_info=True)

# You can also load the 'validation' split if needed
# (ds_validation, _) = tfds.load('food101', split='validation', as_supervised=True)


In [4]:

# Define a function to resize images
def resize_image(image, label):
    image = tf.image.resize(image, (32, 32))
    return image, label

# Apply resizing to the training dataset
ds_train = ds_train.map(resize_image)

# Normalize pixel values to be in the range [0, 1]
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Shuffle and batch the training dataset
ds_train = ds_train.shuffle(10000).batch(64)

# Convert labels to categorical format
def categorical_labels(images, labels):
    return images, tf.one_hot(labels, depth=101)  # 101 classes in Food101 dataset

ds_train = ds_train.map(categorical_labels)

# Verify the shapes
for images, labels in ds_train.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

    
    

Images shape: (64, 32, 32, 3)
Labels shape: (64, 101)


In [5]:
# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(101, activation='softmax'))  # 101 classes in Food101 dataset
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


94781440/94765736 [==============================] - 1s 0us/step


In [6]:
model.fit(ds_train, batch_size=64, epochs=50)

Epoch 1/50
1184/1184 [==============================] - 2428s 2s/step - loss: 4.9420 - accuracy: 0.0173
Epoch 2/50
1184/1184 [==============================] - 2285s 2s/step - loss: 4.3828 - accuracy: 0.0418
Epoch 3/50
1184/1184 [==============================] - 2289s 2s/step - loss: 4.1092 - accuracy: 0.0777
Epoch 4/50
1184/1184 [==============================] - 2293s 2s/step - loss: 3.8758 - accuracy: 0.1131
Epoch 5/50
1184/1184 [==============================] - 2302s 2s/step - loss: 3.6562 - accuracy: 0.1465
Epoch 6/50
1184/1184 [==============================] - 2297s 2s/step - loss: 3.4824 - accuracy: 0.1768
Epoch 7/50
1184/1184 [==============================] - 2293s 2s/step - loss: 3.3330 - accuracy: 0.2013
Epoch 8/50
1184/1184 [==============================] - 2297s 2s/step - loss: 3.1869 - accuracy: 0.2261
Epoch 9/50
1184/1184 [==============================] - 2298s 2s/step - loss: 3.0452 - accuracy: 0.2538
Epoch 10/50
1184/1184 [==============================] - 2305s 2

In [7]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Define a function to resize and normalize test images
def preprocess_test_image(image, label):
    image = tf.image.resize(image, (32, 32))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Apply preprocessing to the test dataset
ds_test = ds_test.map(preprocess_test_image)

# Create a list to store true labels
true_labels = []

# Create a list to store predicted labels
predicted_labels = []

# Make predictions on the test dataset
for image, label in ds_test:
    image = np.expand_dims(image, axis=0)  # Add a batch dimension
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction, axis=1)
    
    true_labels.append(label.numpy())
    predicted_labels.append(predicted_label)

# Convert lists to arrays for evaluation
true_labels = np.concatenate(true_labels)
predicted_labels = np.concatenate(predicted_labels)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print(f"Test Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)


AttributeError: 'DatasetInfo' object has no attribute 'map'

In [8]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


ValueError: Failed to find data adapter that can handle input: <class 'tensorflow_datasets.core.dataset_info.DatasetInfo'>, <class 'NoneType'>